# Multilayer Perceptron (MLP) for Battery Management System (BMS) SOH Estimation

<img src="../../doc/img/MachineLearningNetwork.png" height="1080" width="1920"
     alt="Machine Learning Network"
     style="fit: left; margin-right: 10px;"  />

# Importing Libraries

In [14]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras.layers import Activation 
from tensorflow.keras import layers, Input
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split

# Import Data

In [15]:
# Import Data
df = pd.read_csv('../../res/model_data/batemo_model_data.csv')
df.describe()

,V,I,SOC,T_surf,SOH
count,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06,1.640946e+06
mean,3.471013e+00,3.610140e+00,4.188879e+01,5.180549e+01,8.566437e+01
std,8.445745e-01,2.830966e+01,4.022232e+01,2.451698e+01,9.026790e+00
min,-9.394656e-01,-5.500295e+01,-5.753759e+00,2.499965e+01,7.000000e+01
25%,3.144859e+00,-1.900000e+01,1.230206e+00,2.583835e+01,7.800000e+01
50%,3.693897e+00,5.000000e+00,3.474387e+01,4.826449e+01,8.600000e+01
75%,4.064538e+00,2.600000e+01,8.456010e+01,7.304868e+01,9.400000e+01
max,4.723157e+00,5.500289e+01,1.070423e+02,1.015903e+02,1.000000e+02


# Data Preprocessing

We have to add new columns to the dataset that are the previous values of V, I and T. This will allow our model to detect the trend/gradient of the data.

In [31]:
# Data Preprocessing
# Round the I column for simplicity
df['I_round'] = df['I'].round(0)

# Define the columns to shift
cols_to_shift = ['V', 'I', 'T_surf']

# Apply the shift to the entire dataframe, but only on each unique SOH, I Pair
for col in cols_to_shift:
    df[col + '-1'] = df.groupby(['SOH', 'I_round'])[col].shift(1)

# Drop rows with NaN values and I_round column
df = df.dropna()
df = df.drop(columns=['I_round'])
df

,V,I,SOC,T_surf,SOH,V-1,I-1,T_surf-1
32,3.979128,-40.001522,100.000000,25.000000,100,3.979232,-39.994236,25.000000
33,3.979068,-40.000711,100.000000,25.000000,100,3.979128,-40.001522,25.000000
34,3.979010,-39.999801,100.000000,25.000000,100,3.979068,-40.000711,25.000000
35,3.978947,-39.999732,100.000000,25.000000,100,3.979010,-39.999801,25.000000
36,3.978387,-39.999976,99.999999,25.000000,100,3.978947,-39.999732,25.000000
...,...,...,...,...,...,...,...,...
1640941,3.040704,-41.000000,14.330671,95.558472,70,3.060165,-41.000000,94.628477
1640942,3.019722,-41.000000,13.112293,96.489709,70,3.040704,-41.000000,95.558472
1640943,2.985720,-41.000000,11.355930,97.840139,70,3.019722,-41.000000,96.489709
1640944,2.945987,-41.000000,9.599568,99.207108,70,2.985720,-41.000000,97.840139


# Model Training

In [32]:
# Model Training

# Model Training with regularization and dropout
model = Sequential([
    # input layer
    Input(shape=(6, )),

    # dense layer 1 with L1 regularization
    Dense(256, activation='relu', kernel_regularizer=l1(0.001)),

    # dropout layer
    Dropout(0.5),

    # dense layer 2 with L2 regularization
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),

    # dropout layer
    Dropout(0.5),

    # output layer
    Dense(101, activation='relu'),
])


# Model Testing

In [33]:
# Model Testing
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# define training variables V, I, and T here: 
X = df[['V', 'I', 'T_surf', 'V-1', 'I-1', 'T_surf-1']]
Y = df[['SOH']]

# split up the data into training and testing 
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model.fit(x_train, y_train, epochs=10, 
		batch_size=2000, 
		validation_split=0.2) 

results = model.evaluate(x_test, y_test, verbose = 0) 
print('test loss, test acc:', results)


Epoch 1/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0332 - loss: 4.8415 - val_accuracy: 0.0299 - val_loss: 3.9516
Epoch 2/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0318 - loss: 4.0125 - val_accuracy: 0.0329 - val_loss: 3.9247
Epoch 3/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0334 - loss: 3.9460 - val_accuracy: 0.0328 - val_loss: 3.9013
Epoch 4/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0323 - loss: 3.9004 - val_accuracy: 0.0328 - val_loss: 3.8919
Epoch 5/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0328 - loss: 3.8911 - val_accuracy: 0.0318 - val_loss: 3.8850
Epoch 6/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0325 - loss: 3.7201 - val_accuracy: 0.0294 - val_loss: 3.4398
Epoch 7/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.0311 - loss: 3.4390 - val_accuracy: 0.0294 - val_loss: 3.4388
Epoch 8/10
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0339 - loss: 3.4398 - val_accura

# Model Export

In [4]:
# Model Export
# save Keras model
model.save("model_file_name" +'.h5')
# convert Keras model to a tflite model 
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
with open("model_file_name" + '.tflite', 'wb') as f:
    f.write(tflite_model)
	